<a href="https://colab.research.google.com/github/radeonthe1st/USAL_projects/blob/master/Visualizaci%C3%B3n_Interactiva_de_Incendios_Forestales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualización de incendios forestales

En esta Jupyter Notebook, se muestra una aplicación del módulo *Altair* para la visualización de datos a un dataset de incendios forestales en el parque natural de Montesinhos, en la región Trás-Os-Montes, en el norte de Portugal, cuyo mapa se muestra a continuación, dado que será de relevancia para la visualización.

![Mapa del parque natural de Montesinhos](https://www.researchgate.net/profile/Paulo_Cortez3/publication/238767143/figure/fig1/AS:298804772392991@1448252017812/The-map-of-the-Montesinho-natural-park.png)

---
##Importado de módulos y datos

El primer paso para el desarrollo del programa es importar los módulos necesarios, así como subir el archivo .csv con el que se va a trabajar y cargarlo en una base de datos de pandas.


In [0]:
import altair as alt
import pandas as pd
from google.colab import files
uploaded = files.upload()
#Archivo CSV disponible en http://bit.ly/anavis_csv
import io
fire = pd.read_csv(io.StringIO(uploaded['forestfires.csv'].decode('utf-8')))

Saving forestfires.csv to forestfires.csv


Una vez hecho esto, se definen los métodos de selección que se van a utilizar para la visualización. En este caso, serán un menú desplegable y una selección de caja. Para manejar esta última, será necesario pinchar y arrastrar con el ratón **mientras se mantiene pulsada la tecla Shift**. Además, se declaran los valores de los colores y la opacidad que se va a utilizar en las gráficas.

In [0]:
drop = alt.binding_select(options=['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec',])
brush = alt.selection_interval(
    on="[mousedown[event.shiftKey], mouseup] > mousemove",
    name='brush')
select = alt.selection_single(fields=['month'], bind=drop, name='Mes')
color = alt.condition(select,
                    alt.Color('temp:Q', scale = alt.Scale(scheme='goldorange')),
                    alt.value('lightgray'))
opacity = alt.condition(select, alt.value(0.5), alt.value(0.01))

### Construcción de las gráficas

Se crea la gráfica principal, la cual mostrará en los ejes X e Y la posición en la que se produjo un incendio, y codificará en el color la temperatura media del día en el que ocurrió; y en el tamaño del marcador el número de hectáreas de bosque quemado. Los datos mostrados inicialmente abarcan toda la base de datos, pero al seleccionar un mes del menú desplegable, se mostrarán resaltados todos los incendios pertenecientes al mes en cuestión, recoloreando los demás a un gris claro y reduciendo drásticamente su opacidad.

Además, al pasar el ratón por encima de cada punto, se despliega una ventana que proporciona información adicional sobre el evento, como la velocidad del viento, las precipitaciones, o diversos índices del Forest Fire Weather Index (FWI), creados por el gobierno canadiense y que clasifica diversos parámetros relevantes al riesgo de incendio:

*   FFMC: Factor que mide la inflamabilidad de los detritos en la superficie del suelo
*   DMC: Factor que mide la inflamabilidad de los detritos en capas más profundas del suelo.
*   DC: Factor que mide la sequedad de los materiales de la zona.
*   ISI: Factor de estimación inicial de la probabilidad de extensión del fuego.



In [0]:
mappoints = alt.Chart(fire).mark_point().encode(
x=alt.X('X', scale=alt.Scale(domain=(0,10))),
y=alt.Y('Y', sort='descending',scale=alt.Scale(domain=(0,10))),
color = color,
opacity = opacity,
size='area',
tooltip=['temp','wind','rain','area','FFMC','DMC','DC','ISI']
).add_selection(select,brush)

A continuación, se crea una gráfica secundaria de apoyo en la que, cuando se haga una selección de caja en la principal, se mostrará un histograma del número de eventos recogidos para distintas temperaturas. De la misma forma que en la principal, al pasar el ratón por encima de cada barra, se mostrarán diversos parámetros pertinentes, como el mes en el que se produjo el incendio o el área que abarcó. Esta selección se verá afectada por la selección de mes del menú desplegable en la gráfica primaria.

In [0]:
hist = alt.Chart(fire).mark_bar().encode(
    x=alt.X('temp',axis=alt.Axis(title='Temperatura')),
    y=alt.Y('count(temp):Q',axis=alt.Axis(title='Eventos')),
    color = color,
    opacity = alt.value(0.5),
    tooltip=['X','Y','month','area','temp']
).transform_filter(brush).transform_filter(select)

Finalmente, una vez estén creadas ambas gráficas, sólo resta representarlas gráficamente.

In [0]:
mappoints|hist

alt.HConcatChart(...)